### Load libs

In [3]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
import re
from scipy import spatial
import time
import multiprocessing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhiti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load data

In [4]:
df1=pd.read_excel(r"datasets/extremism.xlsx")

df=pd.DataFrame()
df['id'] = df1['id']
df['text'] = df1['text']
df['class'] = df1['class']
df = df.dropna()

In [5]:
df

,id,text,class
0,1,Таким образом никто не внес большего вклад...,3.0
1,2,"Можно с уверенностью сказать, что все полезное...",3.0
2,3,Славяне должны на нас работать. Если они нам б...,3.0
3,4,"Русское государство росло, развивалось из свои...",3.0
4,5,Поставим власть под контроль народа,3.0
...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0
305,306,Рост продуктивности человеческой деятельности ...,0.0
306,307,"Социальная дифференциация, в том числе по этни...",0.0
307,308,Демографический рост и расширение экономическо...,0.0


### Load models

In [88]:
w2v_model = Word2Vec(
    min_count=10,
    window=10,
    size=300,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

exp_model = w2v_model.wv.load_word2vec_format(r'models/exp_model.bin', binary=True)

In [89]:
exp_model.most_similar('кошка')

KeyError: "word 'кошка' not in vocabulary"

## Prepare funcs

### Tokenization

In [90]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def comma2space(word):
    new_word = ''
    try:
        for char in word:
            if char in patterns:
                char = ' '
            new_word += char
    except TypeError:
        pass
    return new_word
    
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            token = token.strip()
            tokens.append(token)
    return tokens

def dropstopwords(docs):
    stopwords = nltk.corpus.stopwords.words('russian')
    sen =  []
    for doc in docs:
        if (doc not in stopwords):
            sen.append(doc)
    return sen

def dropcomma(doc):
    new_sen = ''
    for char in doc:
        if (char not in patterns):
            new_sen += char
    return new_sen

def tokenize(docs):
    docs = comma2space(docs)
    docs = lemmatize(docs)
    docs = dropstopwords(docs)
    return docs

### Medium word-vector

In [163]:
def mediumVector(sen):
    medVec = np.zeros(300).reshape((1, 300))
    count = 0
    for word in sen:
        try:
            vec = exp_model.word_vec(word)
            medVec += vec
            count += 1
        except:
            pass
    if count >= 0 :
        medVec = medVec / count
    return medVec

## Organise data 

In [164]:
data=df['text'].apply(tokenize)
data

0      [таким, образом, никто, внес, большего, вклада...
1      [уверенностью, сказать, полезное, хорошее, сов...
2      [славяне, должны, работать, нам, нужны, могут,...
3      [русское, государство, росло, развивалось, сво...
4                   [поставим, власть, контроль, народа]
                             ...                        
304    [социальное, соответствующий, подход, предпола...
305    [рост, продуктивности, человеческой, деятельно...
306    [социальная, дифференциация, числе, этническом...
307    [демографический, рост, расширение, экономичес...
308    [соответственно, политика, возникает, связи, у...
Name: text, Length: 309, dtype: object

In [165]:
df['tokenized']=df['text'].apply(tokenize)
df['medVec']=df['tokenized'].apply(mediumVector)
df

,id,text,class,tokenized,medVec
0,1,Таким образом никто не внес большего вклад...,3.0,"[таким, образом, никто, внес, большего, вклада...","[[-0.08076643198728561, -0.03717906400561333, ..."
1,2,"Можно с уверенностью сказать, что все полезное...",3.0,"[уверенностью, сказать, полезное, хорошее, сов...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2,3,Славяне должны на нас работать. Если они нам б...,3.0,"[славяне, должны, работать, нам, нужны, могут,...","[[-0.04596193755666415, -0.04917717600862185, ..."
3,4,"Русское государство росло, развивалось из свои...",3.0,"[русское, государство, росло, развивалось, сво...","[[-0.1227731704711914, -0.11972755752503872, -..."
4,5,Поставим власть под контроль народа,3.0,"[поставим, власть, контроль, народа]","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
...,...,...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0.0,"[социальное, соответствующий, подход, предпола...","[[-0.09335160627961159, -0.05264603905379772, ..."
305,306,Рост продуктивности человеческой деятельности ...,0.0,"[рост, продуктивности, человеческой, деятельно...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
306,307,"Социальная дифференциация, в том числе по этни...",0.0,"[социальная, дифференциация, числе, этническом...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
307,308,Демографический рост и расширение экономическо...,0.0,"[демографический, рост, расширение, экономичес...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."


In [6]:
df['class'] = df['class'].astype('int32')

In [7]:
df

,id,text,class
0,1,Таким образом никто не внес большего вклад...,3
1,2,"Можно с уверенностью сказать, что все полезное...",3
2,3,Славяне должны на нас работать. Если они нам б...,3
3,4,"Русское государство росло, развивалось из свои...",3
4,5,Поставим власть под контроль народа,3
...,...,...,...
304,305,Социальное. Соответствующий подход предполагае...,0
305,306,Рост продуктивности человеческой деятельности ...,0
306,307,"Социальная дифференциация, в том числе по этни...",0
307,308,Демографический рост и расширение экономическо...,0


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df['text'], df['class'], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
y_train

276    0
29     3
247    0
258    0
149    1
      ..
62     3
240    0
143    1
121    1
129    1
Name: class, Length: 222, dtype: int32

In [10]:
from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров

In [11]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

In [48]:
pred = knb_ppl_clf.predict(['Русские вперед убивай нерусских', 'Сегодня хорошая погода', 'Винда', "В чем смысл жизни", "Режьте ебаных чурок", "Хочу свалить в Татарстан"])
print(pred)

[3 1 0 1 3 0]


In [14]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        21
           1       0.80      0.80      0.80        15
           3       0.82      0.90      0.86        20

    accuracy                           0.88        56
   macro avg       0.87      0.87      0.87        56
weighted avg       0.88      0.88      0.88        56



In [15]:
predicted_sgd = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       0.95      0.82      0.88        22
           1       0.93      0.58      0.72        24
           3       0.45      1.00      0.62        10

    accuracy                           0.75        56
   macro avg       0.78      0.80      0.74        56
weighted avg       0.85      0.75      0.76        56



In [16]:
parameters = { 
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              'tfidf__strip_accents':['ascii', 'unicode', None],
               'tfidf__ngram_range':[(1,2), (2,3), (3,4)]
              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train, y_train)
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.828327922077922 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': 'elasticnet', 'tfidf__ngram_range': (1, 2), 'tfidf__strip_accents': None}


In [26]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('sgd_clf', SGDClassifier(penalty='elasticnet'))])
sgd_ppl_clf.fit(X_train, y_train)
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       1.00      0.86      0.93        22
           1       0.87      0.65      0.74        20
           3       0.64      1.00      0.78        14

    accuracy                           0.82        56
   macro avg       0.83      0.84      0.82        56
weighted avg       0.86      0.82      0.82        56



In [27]:
from sklearn.ensemble import RandomForestClassifier
forest = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('forest_clf', RandomForestClassifier())])


In [29]:
forest.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('forest_clf', RandomForestClassifier())])

In [42]:
forest.predict(['Русский язык - самый могучий язык в мире', 'Слава нации, смерть врагам', 'Жизнь прекрасна', 'Белорусские власти объявили забастовку', 'Привет'])

array([3, 3, 3, 3, 3])

In [38]:
predicted_forest = forest.predict(X_test)
print(metrics.classification_report(predicted_forest, y_test))

              precision    recall  f1-score   support

           0       0.42      1.00      0.59         8
           1       0.47      0.70      0.56        10
           3       0.95      0.55      0.70        38

    accuracy                           0.64        56
   macro avg       0.61      0.75      0.62        56
weighted avg       0.79      0.64      0.66        56



In [39]:
forest.score(X_test, y_test)

0.6428571428571429

In [40]:
sgd_ppl_clf.score(X_test, y_test)

0.8214285714285714

In [41]:
knb_ppl_clf.score(X_test, y_test)

0.75